# Infrastructure Setup

In [ ]:
import os
from chi import context

# We select which chameleon site we are using and choose the project
context.use_site("KVM@TACC")
context.choose_project()
username = os.environ.get('USER').replace("_", "-")

## Creating a 1-day lease

In [ ]:
from chi import lease, server, network
from datetime import timedelta

In [ ]:
instance_count = 3
flavor = server.get_flavor('m1.medium')

In [ ]:
mpi_lease = lease.Lease(f"{username}-mpi-lease", duration=timedelta(days=1))
mpi_lease.add_flavor_reservation(id=flavor.id, amount=instance_count)
mpi_lease.submit(idempotent=True)

In [ ]:
# Add a security group, required for KVM
sg = network.SecurityGroup({"name": "MPI-SSH-Access", "description": "Allow inbound SSH traffic to MPI instance"})
sg.add_rule("ingress", "tcp", 22)
sg.submit(idempotent=True)

In [ ]:
reserved_flavor = f"reservation:{mpi_lease.flavor_reservations[0]['id']}"

In [ ]:
mpi_master=server.Server(
        f"{username}-mpi-master",
        image_name="Ubuntu22.04-HPC-MPI-Spack",
        flavor_name=reserved_flavor,
        network_name="sharednet1"
)
mpi_master_hostname = mpi_master.name
mpi_master.submit(idempotent=True)

mpi_workers = []
mpi_worker_hostnames = []
for i in range(instance_count - 1):
    mpi_worker=server.Server(
            f"{username}-mpi-worker-{i+1}",
            image_name="Ubuntu22.04-HPC-MPI-Spack",
            flavor_name=reserved_flavor,
            network_name="sharednet1"
    )
    mpi_worker.submit(idempotent=True)
    mpi_workers.append(mpi_worker)
    mpi_worker_hostnames.append(mpi_worker.name)

In [ ]:
all_hostnames = mpi_master_hostname + "," + ",".join(mpi_worker_hostnames)

In [ ]:
fip = mpi_master.get_floating_ip()
if not fip:
    mpi_master.associate_floating_ip()
    fip = mpi_master.get_floating_ip()

In [ ]:
# Execute only once
mpi_master.add_security_group(sg.id)

## Create inventory.ini to work with ansible

In [ ]:
with open("./inventory.ini", "w") as f:
    f.write("[master_node]\n")
    f.write(f"{mpi_master.name} ansible_host={fip}\n\n")
    
    f.write("[worker_nodes]\n")
    f.write("\n".join(f"{w.name} ansible_host={w.addresses['sharednet1'][0]['addr']}" for w in mpi_workers))
    f.write("\n\n")
    f.write("[worker_nodes:vars]\n")
    f.write(f"ansible_ssh_common_args='-o ProxyJump=cc@{fip}'")

## Use Ansible to create an MPI Cluster

In [ ]:
import ansible_runner
import tempfile
tmpdir = tempfile.TemporaryDirectory()
ansible_run = ansible_runner.run(
    private_data_dir=tmpdir.name,
    inventory=os.path.abspath("inventory.ini"),
    envvars = {
        "ANSIBLE_PYTHON_INTERPRETER": "/usr/bin/python3",
        "ANSIBLE_SSH_ARGS": f"-F {os.path.abspath("config")}",
    },
    extravars={
        "spack_packages": ["pdsh"]
    },
    playbook=os.path.abspath("mpi-cluster.yml"),
    verbosity=0
)

In [ ]:
mpi_master.upload("./examples/src/hello.c", "/home/cc/hello.c")
mpi_master.upload("./examples/mpi_jobs/run_hello.sh", "/home/cc/run_hello.sh")

In [ ]:
mpi_master.execute(f'bash -lc "source /home/cc/run_hello.sh {mpi_master_hostname} {",".join(mpi_worker_hostnames)}"')

### Water_GMX50_bare

It is a prebuilt benchmark test provided by GROMACS. It contains a simulation of bulk water molecules using the TIP3P model

In [ ]:
mpi_master.upload("./examples/mpi_jobs/run_gromacs_water.sh", "/home/cc/run_gromacs_water.sh")

In [ ]:
mpi_master.execute(f'bash -lc "source /home/cc/run_gromacs_water.sh {mpi_master_hostname} {",".join(mpi_worker_hostnames)}"')